In [1]:
import nest_asyncio
nest_asyncio.apply()
import pandas as pd

In [2]:
import stan

schools_code = """
data {
  int<lower=0> J;         // number of schools
  real y[J];              // estimated treatment effects
  real<lower=0> sigma[J]; // standard error of effect estimates
}
parameters {
  real mu;                // population treatment effect
  real<lower=0> tau;      // standard deviation in treatment effects
  vector[J] eta;          // unscaled deviation from mu by school
}
transformed parameters {
  vector[J] theta = mu + tau * eta;        // school treatment effects
}
model {
  target += normal_lpdf(eta | 0, 1);       // prior log-density
  target += normal_lpdf(y | theta, sigma); // log-likelihood
}
"""

In [3]:
schools_data = {"J": 8,
                "y": [28,  8, -3,  7, -1,  1, 18, 12],
                "sigma": [15, 10, 16, 11,  9, 11, 10, 18]}

In [4]:
posterior = stan.build(schools_code, data=schools_data, random_seed=1);


Building...



Building: found in cache, done.Messages from stanc:
Warning in '/var/folders/yb/ztdy95g159n5pnhqz_3qspk40000gn/T/httpstan_fgbjk6sg/model_jszla5rw.stan', line 4, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/var/folders/yb/ztdy95g159n5pnhqz_3qspk40000gn/T/httpstan_fgbjk6sg/model_jszla5rw.stan', line 5, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positiv

In [7]:
%%capture
fit = posterior.sample(num_chains=4, num_samples=1000)


In [8]:
eta = fit["eta"]  # array with shape (8, 4000)
df = fit.to_frame()
df

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,mu,tau,eta.1,...,eta.7,eta.8,theta.1,theta.2,theta.3,theta.4,theta.5,theta.6,theta.7,theta.8
draws,,,,,,,,,,,,,,,,,,,,,
0,-40.355737,0.947192,0.329967,4.0,15.0,0.0,46.986597,4.776580,0.491936,-1.671852,...,-0.549413,0.337620,3.954135,4.974377,4.768906,5.070939,5.102814,5.037204,4.506304,4.942667
1,-36.294045,0.997171,0.278312,4.0,15.0,0.0,43.074367,12.308876,9.406850,1.313661,...,-0.447162,0.539835,24.666287,2.559491,6.279957,8.628074,4.059187,12.070363,8.102492,17.387024
2,-43.327070,0.966449,0.343254,3.0,7.0,0.0,46.386917,10.015984,0.793553,-0.773177,...,1.767242,-0.328516,9.402427,9.150504,10.803257,9.652754,9.660609,8.102712,11.418385,9.755289
3,-40.013120,0.979424,0.366065,3.0,15.0,0.0,42.610731,-1.263474,10.336333,0.420218,...,0.674550,-1.206781,3.080040,10.709243,15.050437,6.530482,-4.848511,6.844603,5.708897,-13.737160
4,-43.137354,0.884152,0.329967,4.0,15.0,0.0,45.879389,-0.881240,0.447448,-2.256127,...,-0.188980,0.228045,-1.890740,-0.619245,-0.641883,-0.957120,-0.743065,-0.863652,-0.965799,-0.779202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,-40.566458,0.874076,0.366065,4.0,15.0,0.0,46.249384,7.547422,3.716107,0.790143,...,-0.863542,-1.498248,10.483677,-0.278449,5.868715,6.977447,3.882360,6.391264,4.338410,1.979773
3996,-36.034390,0.368210,0.329967,2.0,3.0,0.0,38.961394,1.063856,8.270384,1.127886,...,0.404315,0.440177,10.391903,4.562775,-6.897789,2.749260,4.000800,0.067921,4.407695,4.704285
3997,-40.881721,0.994462,0.278312,4.0,15.0,0.0,44.563388,0.253292,6.490790,0.029645,...,2.209834,0.327582,0.445709,2.076608,12.490435,2.205724,4.992847,6.596437,14.596857,2.379557
